### การหาสัญญาณซื้อขายแบบ RSI (RSI ตัด OS ขึ้นซื้อ , ตัด OB ลงขาย)

In [1]:
import pandas as pd
import numpy as np
import sig # เป็น mudule/Libary ที่เราเขียนขึ้นมาเอง อยู่ในไฟล์แล้ว! 

def rsi(close,n): # close คือราคาปิด และ n คือระยะเวลา กำหนดเองหรือแนะนำคือ 14 ตามที่จดไว้
    rows = len(close) # หาจำนวนแถว
    change = [0]*rows # คำนวณราคาที่เปลี่ยนเแปลงจากวันก่อนหน้าโดยขั้นตอนแรกให้สร้าง list ของ item ที่มีค่าเป็น 0 ทั้งหมดตามจำนวนแถวที่เราหาได้จากจำนวนวันที่นับมาจาก close
    change[0] = np.nan # แถวแรกให้เป็น nan วันแรกไม่สามารถเทียบกับวันก่อนหน้าได้
    for i in range(1,rows): # เริ่มคำนวณจากแถวที่ 2 จนถึงแถวที่ rows
        change[i] = close[i]-close[i-1] # เอาราคาปิดของวันที่ ปัจจุบัน - วันก่อนหน้า
    changearr = np.array(change) # แปลง list change ให้เป็น array ในชื่อที่เรียกว่า change array แปลงให้เป็น array เพราะต้องการใช้ method where ของ numpy
    # ใช้ method ที่ชื่อว่า where ให้ทุกแถวของ gain มีค่าเป็น 0 มีจำนวนแถวคือ rows
    # ในวันที่มี changearr > 0 หรือในวันที่มีราคาปิดสูงขึ้น เราจะเอาราคาปิดที่สูงขึ้นแทนค่า 0 ใน list
    # gain สนใจเฉพาะวันที่เปลี่ยนแปลงเพิ่มขึ้น
    gain = np.where(changearr>0,changearr,[0]*rows)
    gain[0] = np.nan # gain ของแถวที่ 0 ให้เป็น nan เพราะคำนวณไม่ได้
    # มีจำนวนแถว = rows และทุกๆ item มีค่าเป็น 0
    # แต่ถ้าวันใดที่เงื่อนไขนี้เป็นจริง เราจะเปลี่ยนค่า 0 เป็นค่าของ -changearr
    # ใส่ลบข้างหน้า changearr เพราะเดิมวันที่ค่าของมันน้อยกว่า 0 อยู่แล้ว เราจะต้องเปลี่ยนค่ากลับมาเป็น + ก็คือเอา - มาคูณ - ให้เป็น +
    # หรือก็คือเราต้องการดูว่ามันเปลี่ยนแปลงไปเท่าไหร่ แต่เราไม่ต้องการค่า -
    # loss สนใจวันที่เปลี่ยนแปลงลดลง
    loss = np.where(changearr<0,-changearr,[0]*rows)
    loss[0] = np.nan
    gain = pd.Series(gain) # แปลงตัว array กลับมาเป็น series ใน pandas 
    loss = pd.Series(loss) # แปลงตัว array กลับมาเป็น series ใน pandas
    avg_gain = [0]*rows # คำนวณค่า avg gain คำนวณจาก series gain 
    avg_loss = [0]*rows # คำนวณค่า avg loss คำนวณจาก series loss
    RS = [np.nan]*rows
    RSI = [np.nan]*rows
    # 1 ถึง n+1 ก็คือแถวสอง(แถวแรกเป็น nan)จนถึงแถวที่ n ทำให้ได้ค่าเฉลี่ยของวันที่ 14 ทั้ง gain และ loss
    avg_gain[n] = gain[1:n+1].mean() 
    avg_loss[n] = loss[1:n+1].mean()
    RS[n] = avg_gain[n]/avg_loss[n] # เอา avg gain วันที่ 14 และ avg loss วันที่ 14 มาหารกัน
    RSI[n] = 100-100/(1+RS[n]) # เอา rs มาหา rsi จากสมการในเล็คเชอร์
    for i in range(n+1,rows):
        avg_gain[i] = ((n-1)*avg_gain[i-1]+gain[i])/n
        avg_loss[i] = ((n-1)*avg_loss[i-1]+loss[i])/n
        RS[i] = avg_gain[i]/avg_loss[i]
        RSI[i] = 100-100/(1+RS[i])
    return pd.Series(RSI) # rsi คืนไป ณ จุดที่เรียกใช้

def RSI_Sig(RSI,OS,OB): # ให้กำหนดระดับ oversold และค่า overbought เองได้ แต่แนะนำคือ 30 และ 70 
    rows = len(RSI) # หาความยาวของ series RSI
    signal = [0]*rows # สร้าง series signal เริ่มต้นให้เป็น 0 ทุกแถว
    for i in range(1,rows): # เริ่มจากวันที่ 2 เพราะการตัดกันเริ่มวันที่ 2
        if (RSI[i-1]<=OS) and (RSI[i]>OS): # ในวันก่อนหน้า/RSI[i-1] อยู่ต่ำกว่าหรือเท่ากับเส้น 30 และวันปัจจุบัน/RSI[i] ได้ตัดเส้น 30 ขึ้นแล้ว เป็นสัญญาณซื้อ
            signal[i] = 1
        elif (RSI[i-1]>=OB) and (RSI[i]<OB): # ในวันก่อนหน้า/RSI[i-1] อยู๋สูงกว่าหรือเท่ากับเส้น 70 และวันปัจจุบัน/RSI[i] ได้ตัดเส้น 70 ลงแล้ว เปฺ็นสัญญาณขาย
            signal[i] = -1
    return signal

df = pd.read_csv('data/SCB.csv')
close = df['Close']
RSI = rsi(close,14) # เรียกใช้ function rsi เพื่อคำนวณค่า RS แล้วไปใช้ RSI
signal = RSI_Sig(RSI,30,70) # เรียกใช้ function RSI_Sig เพื่อหาสัญญาณซื้อขาย
df['RSI'] = RSI # เอา RSI มาสร้าง column ใหม่ที่ชื่อว่า RSI เพิ่มเข้าไปใน df
df['Signal'] = sig.transformSig(signal) # เอา Signal มาสร้าง column ใหม่ที่ชื่อว่า Signal เพิ่มเข้าไปใน df
# แปลงสัญญาณโดย ใช้ transformSig เอามาจาก module sig
# การตัดรอบซื้อขายแตกต่างจาก Week11 เพราะเส้นสัญญาณซื้อและขาย ต่างคนละเส้นกัน
# เช่น ตอนแรกอยู่ RSI < 70 เสดแล้วก็ตัดขึ้นเป็น RSI > 70 แต่ทีนี้ก็ตัดลงอีกครั้งหนึ่งแต่ไม่เกิดสัญญาณอะไรเลยจึ่งไม่เกิดการทำอะไร ดังนั้นแบบนี้จึงตัดขึ้นได้อีกรอบ
# หมายความว่าอาจจะเกิดสัญญาณซื้อหลายๆครั้งจนกว่าจะมีสัญญาณขาย
# ดังนั้นเราต้องแปลงสัญญาณ ไปดูใน module sig.py
df.to_csv('data/SCB_RSI.csv') # เอา df มาสร้าง excel ชื่อว่า SCB_RSI

### การหาสัญญาณซื้อขายแบบ Stochastic Oscillator กลยุทธ์ที่ 1 %K ตัด OS ขึ้นซื้อ และตัด OB ลงขาย (SO1)

In [2]:
import pandas as pd
import sig

def so(high,low,close,kperiods,slowperiods): # ใช้ราคาสูงสุด ต่ำสุด และราคาปิด และมีการกำหนดช่วงระยะเวลาคำนวณของ kperiods และ slowperiods
    ll = low.rolling(window=kperiods).min() # หา lowest low ในช่วงระยะเวลาที่กำหนด
    hh = high.rolling(window=kperiods).max() # หา highest high ในช่วงระยะเวลาที่กำหนด
    fK = (close-ll)/(hh-ll)*100 # fast%K = เอาราคาปิดปัจจุบัน - ราคาต่ำสุดในช่วงเวลาที่กำหนด หารด้วย สูงสุด-ต่ำสุด*100
    sK = fK.rolling(window=slowperiods).mean() # เอา fast %K มา smoothing หรือทำให้เรียบด้วย Moving Average
    return sK

def SO1_Sig(sK,OS,OB):
    rows = len(sK)
    signal = [0] * rows
    for i in range(1,rows):
        if (sK[i-1]<=OS) and (sK[i]>OS): # ตอนแรกต่ำกว่า OS วันต่อมาสูงกว่า OS sK ตัด OS ขึ้น เป็นสัญญาณซื้อ
            signal[i] = 1 
        elif (sK[i-1]>=OB) and (sK[i]<OB): # ตอนแรกสูงกว่า OB วันต่อมาต่ำกว่า OB sK ตัด OB ลง เป็นสัญญาณขาย
            signal[i] = -1
    return signal

df = pd.read_csv('data/SCB.csv')
high = df['High']
low = df['Low']
close = df['Close']
sK = so(high,low,close,5,3)
signal = SO1_Sig(sK,20,80) # เอา slow %K หาสัญญาณซื้อขาย
df['slowK'] = sK # คำนวณค่า Slow %K ด้วย  
df['Signal'] = sig.transformSig(signal) # เอา Slow %K มาคำนวณหา signal และเอามา tranform เหมือน RSI 
df.to_csv('data/SCB_SO1.csv')

### การหาสัญญาณซื้อขายแบบ Stochastic Oscillator กลยุทธ์ที่ 2 %K ตัด %D (SO2)

In [3]:
import pandas as pd
import sig

# kperiods ใช้หาค่า fast%K, slowperiods ใช้หาค่า slow%K, dperiods ใช้หาค่า slow%D
def so(high,low,close,kperiods,slowperiods,dperiods): # so = stochastic oschillator
    # คำนวณจากสูตรแต่ละอย่าง
    ll = low.rolling(window=kperiods).min()
    hh = high.rolling(window=kperiods).max()
    fK = (close-ll)/(hh-ll)*100
    sK = fK.rolling(window=slowperiods).mean()
    sD = sK.rolling(window=dperiods).mean()
    return [sK, sD] # ส่งค่า slow%K และ slow%D ไปให้จุดที่เรียกใช้

def SO2_Sig(sK,sD): # รับค่า slow%K และ slow%D ที่คำนวณไว้มาใช้
    rows = len(sK) # หาจำนวนแถวด้วย sK
    signal = [0] * rows # เอา 0 * จำนวณ rows ทั้งหมด
    for i in range(1,rows):
        if (sK[i-1]<=sD[i-1]) and (sK[i]>sD[i]): # sk ตัด sD ขึ้น ตอนแรก sK ต้องน้อยกว่า sD วันต่อมา sK > sD sK ตัด sD ขึ้น
            signal[i] = 1
        elif (sK[i-1]>=sD[i-1]) and (sK[i]<sD[i]): # sK ตัด SD ลง ตอนแรก sK ต้องมากกว่า sD วันต่อมา sK < sD
            signal[i] = -1
    return signal

df = pd.read_csv('data/SCB.csv') # อ่าน SCB.csv เข้ามาใน df
high = df['High'] # สร้าง series ใหม่มาเป็น high
low = df['Low'] # สร้าง series ใหม่มาเป็น low
close = df['Close'] # สร้าง series ใหม่มาเป็น close
[sK, sD] = so(high,low,close,5,3,3) # เรียกใช้ function so ที่สร้างไว้ข้างบน และตั้ง sK และ sD ไว้รับค่า
signal = SO2_Sig(sK,sD)
df['slowK'] = sK
df['slowD'] = sD
df['Signal'] = sig.transformSig(signal) # ไม่เกิดปัญหาไม่ต้อง tranform เพราะตัดเส้นเดียวกัน ไม่ได้ตัด 2 เส้นเหมือนกลยุทธ์ก่อนหน้า
df.to_csv('data/SCB_SO2.csv')

### การหาสัญญาณซื้อขายแบบ Bollinger Bands ตัด LB ขึ้นซื้อ ตัด UB ลงขาย (BB)

In [4]:
import pandas as pd
import sig

def bb(close,periods,nstd): # nstd = number of standard deviation
    mb = close.rolling(window=periods).mean()
    std = close.rolling(window=periods).std()
    ub = mb+nstd*std # +2เท่า std
    lb = mb-nstd*std # -2เท่า std
    return [mb,ub,lb]

def BB_Sig(close,lb,ub):
    rows = len(close)
    signal = [0] * rows
    for i in range(1,rows):
        if (close[i-1]<=lb[i-1]) and (close[i]>lb[i]): # ราคาปิดตัด lower band ขึ้น
            signal[i] = 1
        elif (close[i-1]>=ub[i-1]) and (close[i]<ub[i]): # ราคาปิดตัด Upper band ลง
            signal[i] = -1
    return signal

df = pd.read_csv('data/SCB.csv')
close = df['Close']
# จำนวนวันแนะนำเป็น 20 และ การหาแถบบนและแถบล่างควรจะเป็น 2 เท่าของ STDEV
[mb,ub,lb] = bb(close,20,2) # ส่ง argument close 20(ระยะเวลาที่ใช้คำนวณ) และ 2(2เท่าของ stdv)
signal = BB_Sig(close,lb,ub)
df['UB'] = ub
df['MB'] = mb
df['LB'] = lb
df['Signal'] = sig.transformSig(signal) # ตัด 2 เส้นจึงต้อง tranform
df.to_csv('data/SCB_BB.csv') 